In [1]:
from riptide import *
from cobra.flux_analysis.variability import find_essential_genes, find_essential_reactions
from cobra.medium.minimal_medium import minimal_medium

In [5]:
iCdR703 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR703.json')

In [6]:
iCdR703

Name,iCdG703
Memory address,0x07f2d4e2dac10
Number of metabolites,1243
Number of reactions,1313
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [7]:
# Parse RefSeq gene ID lookup table
refseq_dict = {}
with open('/home/mjenior/Desktop/active_projects/Tamayo/c-di-AMP/CdR20291_genes.tsv', 'r') as gene_catalog:
    header = gene_catalog.readline()
    for line in gene_catalog:
        line = line.split()
        refseq_dict[line[1]] = line[0]

In [19]:
cd196_missing = ['CDR20291_0183','CDR20291_1419','CDR20291_1744','CDR20291_1745','CDR20291_1746',
                 'CDR20291_1747','CDR20291_1748','CDR20291_1749','CDR20291_1750','CDR20291_1751',
                 'CDR20291_1752','CDR20291_1753','CDR20291_1754','CDR20291_1755','CDR20291_1756',
                 'CDR20291_1757','CDR20291_1758','CDR20291_1759','CDR20291_1760','CDR20291_1761',
                 'CDR20291_1762','CDR20291_1763','CDR20291_1764','CDR20291_1765','CDR20291_1766',
                 'CDR20291_1767','CDR20291_1768','CDR20291_1769','CDR20291_1770','CDR20291_1771',
                 'CDR20291_1772','CDR20291_1773','CDR20291_1774','CDR20291_1775','CDR20291_1776',
                 'CDR20291_1777','CDR20291_1778','CDR20291_1779','CDR20291_1780','CDR20291_1781',
                 'CDR20291_1782','CDR20291_1783','CDR20291_1784','CDR20291_1785','CDR20291_1786',
                 'CDR20291_1787','CDR20291_1788','CDR20291_1809']

check_missing = []
for x in cd196_missing:
    try:
        check_missing.append(refseq_dict[x])
    except:
        continue

check_missing = set(check_missing)
included_genes = []
for x in iCdR703.genes:
    if x.id in check_missing:
        included_genes.append(x.id)

print('Genes missing in CD196:', len(included_genes))

Genes missing in CD196: 0


In [5]:
# Read in transcription
solo = {}
with open('/home/mjenior/Desktop/active_projects/Zackular_enterococcus/solo_norm.tsv', 'r') as transcription:
    for line in transcription:
        line = line.split()
        abund = float(line[1])
        try:
            patric = refseq_dict[line[0]]
            solo[patric] = abund
        except:
            continue
        
coculture = {}
with open('/home/mjenior/Desktop/active_projects/Zackular_enterococcus/coculture_norm.tsv', 'r') as transcription:
    for line in transcription:
        line = line.split()
        abund = float(line[1])
        try:
            patric = refseq_dict[line[0]]
            coculture[patric] = abund
        except:
            continue

In [2]:
import cobra

iCdR700 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700.json')

# Ornithine conversion and alanine fermentation

cpd00064_e = iCdR700.metabolites.cpd00064_e # ornithine
cpd00064_c = iCdR700.metabolites.cpd00064_c # ornithine
cpd00001_c = iCdR700.metabolites.cpd00001_c # H20
cpd00002_c = iCdR700.metabolites.cpd00002_c # ATP
cpd00008_c = iCdR700.metabolites.cpd00008_c # ADP
cpd00009_c = iCdR700.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdR700.metabolites.cpd00067_c # H+

rxn05156_c = cobra.Reaction('rxn05156_c')
rxn05156_c.name = 'Ornithine ABC transport'
rxn05156_c.lower_bound = 0.
rxn05156_c.upper_bound = 1000.
rxn05156_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00064_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00064_c: 1.0
})
rxn05156_c.annotation['seed.reaction'] = 'rxn05156'
rxn05156_c.annotation['bigg.reaction'] = ['ORNabc','ORNabcpp']
rxn05156_c.gene_reaction_rule = '645463.3.peg.862 or 645463.3.peg.865'
# str. 630 = or
iCdR700.add_reactions([rxn05156_c])
iCdR700.reactions.rxn05156_c.annotation['sbo'] = 'SBO:0000629'

cpd00129_c = iCdR700.metabolites.cpd00129_c # proline
cpd00013_c = iCdR700.metabolites.cpd00013_c # ammonia

rxn00471_c = cobra.Reaction('rxn00471_c')
rxn00471_c.name = 'Ornithine cyclodeaminase'
rxn00471_c.lower_bound = 0.
rxn00471_c.upper_bound = 1000.
rxn00471_c.add_metabolites({
    cpd00064_c: -1.0,
    cpd00129_c: 1.0,
    cpd00013_c: 1.0,
    cpd00001_c: 1.0
})
rxn00471_c.annotation['seed.reaction'] = 'rxn00471'
rxn00471_c.annotation['kegg.reaction'] = 'R00671'
rxn00471_c.gene_reaction_rule = '645463.3.peg.500 or 645463.3.peg.260'
# str. 630 = or
iCdR700.add_reactions([rxn00471_c])
iCdR700.reactions.rxn00471_c.annotation['sbo'] = 'SBO:0000629'

iCdR700.reactions.rxn14293_c.gene_reaction_rule = '645463.3.peg.500 or 645463.3.peg.260'
iCdR700.reactions.rxn00655_c.gene_reaction_rule= ''

#------------------------------------------------------------------------------#

# Alanine Stickland fermentation pathway

# Alanine --> Pyruvate
cpd00035_c = iCdR700.metabolites.cpd00035_c # alanine
cpd00020_c = iCdR700.metabolites.cpd00020_c # Pyruvate

R00258_c = cobra.Reaction('R00258_c')
R00258_c.name = 'Alanine transaminase'
R00258_c.lower_bound = -1000.
R00258_c.upper_bound = 1000.
R00258_c.add_metabolites({
    cpd00035_c: -1.0,
    cpd00001_c: -1.0,
    cpd00020_c: 1.0,
    cpd00013_c: 1.0,
    cpd00067_c: 1.0
})
R00258_c.gene_reaction_rule = '645463.3.peg.2419'
# str. 630 = 272563.120.peg.2623
iCdR700.add_reactions([R00258_c])
iCdR700.reactions.R00258_c.annotation['sbo'] = 'SBO:0000629'
iCdR700.reactions.R00258_c.annotation['kegg.reaction'] = 'R00258'

# Pyruvate --> Acetyl-CoA (already present: rxn00157_c)
cpd00010_c = iCdR700.metabolites.cpd00010_c # CoA
cpd00022_c = iCdR700.metabolites.cpd00022_c # Acetyl-CoA
cpd00047_c = iCdR700.metabolites.cpd00047_c # Formate

# Acetyl-CoA --> Acetylphosphate
cpd00196_c = iCdR700.metabolites.cpd00196_c # Acetylphosphate

rxn00173_c = cobra.Reaction('rxn00173_c')
rxn00173_c.name = 'Phosphate acetyltransferase'
rxn00173_c.lower_bound = -1000.
rxn00173_c.upper_bound = 1000.
rxn00173_c.add_metabolites({
    cpd00022_c: -1.0,
    cpd00009_c: -1.0,
    cpd00196_c: 1.0,
    cpd00010_c: 1.0
})
rxn00173_c.gene_reaction_rule = '645463.3.peg.125'
# str. 630 = 272563.120.peg.124
iCdR700.add_reactions([rxn00173_c])
iCdR700.reactions.rxn00173_c.annotation['sbo'] = 'SBO:0000629'
iCdR700.reactions.rxn00173_c.annotation['seed.reaction'] = 'rxn00173'
iCdR700.reactions.rxn00173_c.annotation['kegg.reaction'] = 'R00230'
iCdR700.reactions.rxn00173_c.annotation['bigg.reaction'] = ['PTA','PTAr']

# Acetylphosphate --> Acetate (already in place: rxn00225_c)

#------------------------------------------------------------------------------#

# Ornithine conversion to alanine

cpd02448_c = cobra.Metabolite(
    'cpd02448_c',
    formula='C5H13N2O2',
    name='2,4-diaminopentanoate',
    compartment='cytosol')
cpd02448_c.annotation['seed.compound'] = 'cpd02448'
cpd02448_c.annotation['kegg.compound'] = 'C03943'

gene = cobra.Gene('645463.3.peg.416')

# D-ornithine <--> 2,4-diaminopentanoate
rxn01780_c = cobra.Reaction('rxn01780_c')
rxn01780_c.name = 'Ornithine aminomutase'
rxn01780_c.lower_bound = -1000.
rxn01780_c.upper_bound = 1000.
rxn01780_c.add_metabolites({
    cpd00064_c: -1.0,
    cpd02448_c: 1.0
})
rxn01780_c.gene_reaction_rule = '645463.3.peg.416'
# str. 630 = 272563.120.peg.471
iCdR700.add_reactions([rxn01780_c])
iCdR700.reactions.rxn01780_c.annotation['kegg.reaction'] = 'R02461'
iCdR700.reactions.rxn01780_c.annotation['sbo'] = 'SBO:0000629'

iCdR700.genes.get_by_id('645463.3.peg.416').name = 'D-Ornithine 4,5-aminomutase E subunit (EC 5.4.3.5) | oraE'
iCdR700.genes.get_by_id('645463.3.peg.416').annotation['sbo'] = 'SBO:0000243'
iCdR700.genes.get_by_id('645463.3.peg.416').annotation['patric'] = 'fig|645463.3.peg.416'
iCdR700.genes.get_by_id('645463.3.peg.416').annotation['refseq'] = 'YP_003216896.1'
iCdR700.genes.get_by_id('645463.3.peg.416').annotation['refseq.locustag'] = 'CDR20291_0389'
iCdR700.genes.get_by_id('645463.3.peg.416').annotation['ccds'] = 'ATGTTAAAGGAAAATAAAAAATTGGATATAGACTATATATTAAAAGATTTGGATAAATATAAACCAAAACGAAGAGGTTGGGTTTGGAGAGAACACTTGGAAAATCTTGAAATGGGACCTTTTAAATATAAAGACTGTACAAAACCTTTAAAGAAGAGTGTAGGGCTACCATCATCTAAGTATTTCAATAATATAGACCCTCAACCAAGTCCAGTTATTACTACTGAAATAGCTTCTGGTAGATTTGAGGATGATATAAGAAGAATGAGAATGGCAGCTCATCATGGAGCTGACCATCTAATGGTTATCAGAACTGCTGGACAATCACATTTTGATGGGCTTATAGAAGGAACTCCACAAGGTATAGGTGGAGTTCCAATAACAAGAAAACAAGTAAGAGCTCAAAGGAAGGCTCTTGATTTTATAGAAGAAGAAGTTGGTAGACCTATTAATTACCATTCATATGTAAGTGGTGTTGCTGGTCCTGAAGTAGCAGTAATGTTTGCTGAGGAAGGAGTCAATGGAGCACATCAAGACCCACAATATAATGTTCTGTATAGGAATATAAACATGGTAAGGTCTTTTGTAGATGCTTGTGAGGCAAAGAAAATAATGGCTTTTGCCAATATAGCTCAAATTGATGGAGCACATAATGCTAATGCTACTGCTAGAGAAGCATGGAAGGTAATGCCTGAACTTATGGTTCAACACGCTTTAAACTCTATATTCTCAGAAAAAATAGGTATAGATAAAAGTAATATATGCTTATCAACAGTTCCACCAACAGCACCTCCAGCACCATGTTTAAGAATAGACTTACCTTACGCTGTTGCACTTAGAGAATTATTTAGTGATTATAAGATGAGAGCTCAAATGAATACTAAATATATGGAATCATCAACTAGAGAAGCAACAGTTACTCATGTATTAAACTTATTAACATCAGTATTAACTAGAGCTGACATACAATCAACTATAACACCTGATGAAGGTAGAAATGTACCTTGGCATATATACAATATAGAAGCATGTGATACAGCAAAGCAAGCTTTAGTTGGTATGGATGGGCTTATGGATATGATAGAACTTAAAGATGTCGGTGAATTAAGAGACAAAGCAAGAGAACTTAAAGAAAGAGCTGTATTATATATGGAAGAAATATCTGAAGTTGGTGGATATTTTGAATCTGTGGAACAAGGATTTTTTGTTGATTCTGGAAATTATCCAGAAAGAAATGGTGATGGAATATCTAGAAAAATAAAAGGTGGAGTTGGTGAAGGTACTGTTTATGAAAGAGAAGAAGATTATTTAGCACCTGTAACTGCTCACTTTGGATACAATAATGTAGCTCAATATGATGAAAATGCTATAGATAATCCATCAATGTTAATAGATGGATGTACATTTGAAAATCCAGACAAAATAATATATATTGATGAGCTTGACGATTTTGATAATGTAGAAAATAGACTAAAAGAATCATATGAGTACAGAAATGGAACTAAAATTAAGCCTGAAATGGAATGGTGTGCAGATGGAATTGTAATGATAACAATGATGCTTCCTACTGATAAAAGAACAGCAGAGTTTGCAGCTTTGGAATTTGTCAAAAAAATGAACTTACAAGAAATTGAGGTCATAAGTAGAGAAGTTATGCATGAATCTGAAGGGACTAGAATAGAAGTGAAAGGAAGAGTTCCGTTTGATATAGATTTAAACAATTTAGTAATACCAGAAGAACCTAAGGTACTAACTGATGAAGAAATAAGAGCAGATATAGAAGAAAAACCTATGAAAATAGTTTCAGCTACGGTTGGAGAAGATGAACATTCAGTTGGTTTAAGAGAAATAATAGATATAAAACATGGAGGTATTGAAAAATATGGCATAGAATGCCACTATCTAGGGACTTCTGTACCTGTAGAAAAATTGGTAGATGCAGCGATTGAACTGAATGCAGATGCAATACTTGCATCTACAATAATAAGTCATGATGATATTCACTATAAAAACATGAAAAAATTACACGATTATTGTGTAGAAAAAGGTATAAGAGATAAAGTAATGATTGCGTGTGGAGGAACACAAGTAACACCTGAAATAGCTGTTGAACAAGGCATCGATGCTGGATTTGGAAGAAACAGCAAAGGCATACATGTAGCCACTTTCTTAGTTGAAAAAAGAAGAGAAATGAGTAATAAATAA'

cpd24394_c = cobra.Metabolite(
    'cpd24394_c',
    formula='C5H9NO3',
    name='2-amino-4-ketopentanoate',
    compartment='cytosol')
cpd24394_c.annotation['sbo'] = 'SBO:0000247'
cpd24394_c.annotation['seed.compound'] = 'cpd02448'
cpd24394_c.annotation['metacyc'] = 'CPD-299'

cpd00006_c = iCdR700.metabolites.cpd00006_c # NADP
cpd00005_c = iCdR700.metabolites.cpd00005_c # NADPH

# 2,4-diaminopentanoate <--> 2-amino-4-ketopentanoate
rxn27496_c = cobra.Reaction('rxn27496_c')
rxn27496_c.name = '2,4-diaminopentanoate dehydrogenase'
rxn27496_c.lower_bound = -1000.
rxn27496_c.upper_bound = 1000.
rxn27496_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd02448_c: -1.0,
    cpd00006_c: -1.0,
    cpd00013_c: 1.0,
    cpd00067_c: 1.0,
    cpd24394_c: 1.0,
    cpd00005_c: 1.0
})
rxn27496_c.gene_reaction_rule = '645463.3.peg.412'
# str. 630 = 272563.120.peg.467
iCdR700.add_reactions([rxn27496_c])
iCdR700.reactions.rxn01905_c.gene_reaction_rule = ''
iCdR700.reactions.rxn27496_c.annotation['sbo'] = 'SBO:0000629'
iCdR700.reactions.rxn27496_c.annotation['seed.reaction'] = 'rxn27496'

gene2 = cobra.Gene('645463.3.peg.413')
gene3 = cobra.Gene('645463.3.peg.414')

# 2-amino-4-ketopentanoate <--> alanine
rxn19080_c = cobra.Reaction('rxn19080_c')
rxn19080_c.name = '2-amino-4-oxopentanoate thiolase'
rxn19080_c.lower_bound = -1000.
rxn19080_c.upper_bound = 1000.
rxn19080_c.add_metabolites({
    cpd00010_c: -1.0,
    cpd24394_c: -1.0,
    cpd00022_c: 1.0,
    cpd00035_c: 1.0
})
rxn19080_c.gene_reaction_rule = '645463.3.peg.413 and 645463.3.peg.414'
# str. 630 = and
iCdR700.add_reactions([rxn19080_c])
iCdR700.reactions.rxn19080_c.annotation['sbo'] = 'SBO:0000629'
iCdR700.reactions.rxn19080_c.annotation['seed.reaction'] = 'rxn19080'

iCdR700.genes.get_by_id('645463.3.peg.413').name = '2-amino-4-ketopentanoate thiolase alpha subunit [EC:2.3.1.263] | ortA'
iCdR700.genes.get_by_id('645463.3.peg.413').annotation['sbo'] = 'SBO:0000243'
iCdR700.genes.get_by_id('645463.3.peg.413').annotation['patric'] = 'fig|645463.3.peg.413'
iCdR700.genes.get_by_id('645463.3.peg.413').annotation['kegg'] = 'K21399'
iCdR700.genes.get_by_id('645463.3.peg.413').annotation['refseq'] = 'YP_003216893.1'
iCdR700.genes.get_by_id('645463.3.peg.413').annotation['refseq.locustag'] = 'CDR20291_0386'
iCdR700.genes.get_by_id('645463.3.peg.413').annotation['ccds'] = 'ATGGATGCTAAAATTAATGATTGGGTTATTATACACAATATAGTATTGACTCCAGAAGAAAGAGCTCCACAAGTTCCAGAGGATACTAAGAAAGTATCCTTAGAAATGTGGGTTAAAGGTTTTATACAAAGTGATGCTTCTATAGGCGACTTAGTAGAAGTTAAAACAATTACAGGTAGACTTGTAAAAGGTAATTTACTTAAAGTAAATCCATATTATACTCATGATTATGGTAAATGTATACCAGAGTTACTCCAAATAGGAATCCAAGCTAAAGAAATATTATTTGGAGGTGTATATAATGAATAA'

iCdR700.genes.get_by_id('645463.3.peg.414').name = '2-amino-4-ketopentanoate thiolase beta subunit [EC:2.3.1.263] | ortA'
iCdR700.genes.get_by_id('645463.3.peg.414').annotation['sbo'] = 'SBO:0000243'
iCdR700.genes.get_by_id('645463.3.peg.414').annotation['patric'] = 'fig|645463.3.peg.414'
iCdR700.genes.get_by_id('645463.3.peg.414').annotation['kegg'] = 'K21400'
iCdR700.genes.get_by_id('645463.3.peg.414').annotation['refseq'] = 'YP_003216894.1'
iCdR700.genes.get_by_id('645463.3.peg.414').annotation['refseq.locustag'] = 'CDR20291_0387'
iCdR700.genes.get_by_id('645463.3.peg.414').annotation['ccds'] = 'ATGAATAATTCAACATCAATAAAAGATATGAGTTATCAAGCTGTTATGGGAAGAAATAACGAAATAATGAAAAATGCAATCGGATTGGATTATTCTTCTTTTGAACAAGAGGGAATAGGGTTTGATTATGAGAAAATGATGAGTGAAACAGGATATACATTACAAGATATTGAAGCAATTCAATCACAGTATGCTGTTGGAAATACTCCTCTAATAGAGCTTAAAAATTTGACTAAATTAGCTAGAAAATGTGCTAAAGAAGGTAAAGGAGCTAAAATATTTGTAAAAGATGAAGCAATGAATGCATCTGGTAGTTTTAAAGCAAGAAGAGCAGCTACTGCAGTATATCATGCCAAACAAATGGGATATAAAGGTGTAATAGCAGCTACCAGTGGTAACTATGGTGCAGCTGTAGCATCACAAGCGGCAATGCAGGGATTAAAATGTATAATAGTTCAAGAATGTTATGATTCTAATGGTGTAGGTCAACCTGAAATTATAGAAAAAGCACGAAAATGTGAGGCATATGGAGCAGAAGTTGTTCAGTTAACTGTTGGACCAGAGTTATTTTATACGTTCTTAGTTTTATTAGAAGAAACAGGCTATTTTAACGCATCATTATATTCACCATTTGGAATTGCAGGTGTAGAAACTTTAGGTTATGAGTTAGCTATCCAATTTAGAGAAAAGTATAAAAAAGACCCTGATATAGTTGTTTGTACAAATGCAGGTGGTGGTAATCTTACTGGAACTGCAAGAGGATTGATAAAAGCTGGTTCCATTAATACTAAAGTAGTAGGAGCAAGTGTAGATTTAAAAGGATTACACATGGCTAGTGATAATCAATTTAATAAAAAATCATTTACAACAGGACATACAGGTTTTGGGATACCATACTGTACATGGCCTGATAGGTCAGATGTACCTAGGTCTGCAGCTAGACCACTTAGATATATGGATAGATATGTATTAGTAAAACAAGGAGAAGTTTTTTATACAACTGAATTATTGGCACAGCTAGAAGGTATAGAAAGAGGTCCAGCAGGAAATACATCTTTAGCAGCAGCATTCTCTTTAGCTCAAGAGTTAGATGAGGATAAAGTTATAGTGGTACAAGAAACAGAATATACAGGCGCAGGTAAACATATATGTCCACAGCTTACTTTTGCAAGAGAAAATGGAATAGATATAAAATTTGGAAATCCAAGAGAAGAAATTGCAGGAGTAAACTTAATATTACCAGAAAGACCAGAATTATTGAAATGTGTTGATATTGATATGAATAAAATAAGAAAGTCATTTATAAAAAATTGCATATCTAATAATCACATTGATAATATTGATAAGCTAAGTAATCGAGATATAGAATTTTTAATGGAAGAAGTAAAAAGTTCAAGAGATTTTGTTGTTGATGTATTAAATAATTTATAG'

#------------------------------------------------------------------------------#

# Correct p-Cresol metabolism
iCdR700.reactions.rxn07124_c.remove_from_model(remove_orphans=True)
iCdR700.reactions.ID008_c.remove_from_model(remove_orphans=True)
iCdR700.reactions.EX_cpd03170_e.remove_from_model(remove_orphans=True)
iCdR700.reactions.ID004_c.remove_from_model(remove_orphans=True)

iCdR700.reactions.rxn05061_c.gene_reaction_rule = '645463.3.peg.167 and 645463.3.peg.168 and 645463.3.peg.169'

iCdR700.reactions.ID005_c.id = 'rxn29230_c'
iCdR700.reactions.rxn29230_c.annotation['seed.reaction'] = 'rxn29230'
iCdR700.reactions.ID009_c.id = 'rxn08184_c'
iCdR700.reactions.rxn08184_c.annotation['seed.reaction'] = 'rxn08184'

# Correct phenylpropanoate transport
iCdR700.reactions.ID007_c.remove_from_model(remove_orphans=False)
cpd03343_c = iCdR700.metabolites.cpd03343_c # Phenylpropanoate
cpd03343_e = iCdR700.metabolites.cpd03343_e # Phenylpropanoate

rxn10133_c = cobra.Reaction('rxn10133_c')
rxn10133_c.name = 'Phenylpropanoate hydrogen symport'
rxn10133_c.lower_bound = -1000.
rxn10133_c.upper_bound = 1000.
rxn10133_c.add_metabolites({
    cpd00067_c: -1.0,
    cpd03343_c: -1.0,
    cpd00067_c: 1.0,
    cpd03343_e: 1.0
})
iCdR700.add_reactions([rxn10133_c])
iCdR700.reactions.rxn10133_c.annotation['sbo'] = 'SBO:0000629'
iCdR700.reactions.rxn10133_c.annotation['seed.reaction'] = 'rxn10133'
iCdR700.reactions.rxn10133_c.annotation['bigg.reaction'] = ['PPPNt2','PPPNt2r','PPPNt2rpp']


#cobra.io.save_json_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700_v1.1.sbml')
#cobra.io.write_sbml_model(iCdR700, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR700_v1.1.json')


In [14]:

iCdG698 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG698.json')

# Ornithine conversion and alanine fermentation

cpd00064_e = iCdG698.metabolites.cpd00064_e # ornithine
cpd00064_c = iCdG698.metabolites.cpd00064_c # ornithine
cpd00001_c = iCdG698.metabolites.cpd00001_c # H20
cpd00002_c = iCdG698.metabolites.cpd00002_c # ATP
cpd00008_c = iCdG698.metabolites.cpd00008_c # ADP
cpd00009_c = iCdG698.metabolites.cpd00009_c # Phosphate
cpd00067_c = iCdG698.metabolites.cpd00067_c # H+

rxn05156_c = cobra.Reaction('rxn05156_c')
rxn05156_c.name = 'Ornithine ABC transport'
rxn05156_c.lower_bound = 0.
rxn05156_c.upper_bound = 1000.
rxn05156_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd00002_c: -1.0,
    cpd00064_e: -1.0,
    cpd00008_c: 1.0,
    cpd00009_c: 1.0,
    cpd00067_c: 1.0,
    cpd00064_c: 1.0
})
rxn05156_c.annotation['seed.reaction'] = 'rxn05156'
rxn05156_c.annotation['bigg.reaction'] = ['ORNabc','ORNabcpp']
rxn05156_c.gene_reaction_rule = '272563.120.peg.897 or 272563.120.peg.900'
iCdG698.add_reactions([rxn05156_c])
iCdG698.reactions.rxn05156_c.annotation['sbo'] = 'SBO:0000629'

cpd00129_c = iCdG698.metabolites.cpd00129_c # proline
cpd00013_c = iCdG698.metabolites.cpd00013_c # ammonia

rxn00471_c = cobra.Reaction('rxn00471_c')
rxn00471_c.name = 'Ornithine cyclodeaminase'
rxn00471_c.lower_bound = 0.
rxn00471_c.upper_bound = 1000.
rxn00471_c.add_metabolites({
    cpd00064_c: -1.0,
    cpd00129_c: 1.0,
    cpd00013_c: 1.0,
    cpd00001_c: 1.0
})
rxn00471_c.annotation['seed.reaction'] = 'rxn00471'
rxn00471_c.annotation['kegg.reaction'] = 'R00671'
rxn00471_c.gene_reaction_rule = '272563.120.peg.553'
# str. 630 = or
iCdG698.add_reactions([rxn00471_c])
iCdG698.reactions.rxn00471_c.annotation['sbo'] = 'SBO:0000629'

iCdG698.reactions.rxn14293_c.gene_reaction_rule = '272563.120.peg.553'
# str. 630 = 272563.120.peg.553 or
iCdG698.reactions.rxn00655_c.gene_reaction_rule= ''

#------------------------------------------------------------------------------#

# Alanine Stickland fermentation pathway

# Alanine --> Pyruvate
cpd00035_c = iCdG698.metabolites.cpd00035_c # alanine
cpd00020_c = iCdG698.metabolites.cpd00020_c # Pyruvate

R00258_c = cobra.Reaction('R00258_c')
R00258_c.name = 'Alanine transaminase'
R00258_c.lower_bound = -1000.
R00258_c.upper_bound = 1000.
R00258_c.add_metabolites({
    cpd00035_c: -1.0,
    cpd00001_c: -1.0,
    cpd00020_c: 1.0,
    cpd00013_c: 1.0,
    cpd00067_c: 1.0
})
R00258_c.gene_reaction_rule = '272563.120.peg.2623'
iCdG698.add_reactions([R00258_c])
iCdG698.reactions.R00258_c.annotation['sbo'] = 'SBO:0000629'
iCdG698.reactions.R00258_c.annotation['kegg.reaction'] = 'R00258'

# Pyruvate --> Acetyl-CoA (already present: rxn00157_c)
cpd00010_c = iCdG698.metabolites.cpd00010_c # CoA
cpd00022_c = iCdG698.metabolites.cpd00022_c # Acetyl-CoA
cpd00047_c = iCdG698.metabolites.cpd00047_c # Formate

# Acetyl-CoA --> Acetylphosphate
cpd00196_c = iCdG698.metabolites.cpd00196_c # Acetylphosphate

rxn00173_c = cobra.Reaction('rxn00173_c')
rxn00173_c.name = 'Phosphate acetyltransferase'
rxn00173_c.lower_bound = -1000.
rxn00173_c.upper_bound = 1000.
rxn00173_c.add_metabolites({
    cpd00022_c: -1.0,
    cpd00009_c: -1.0,
    cpd00196_c: 1.0,
    cpd00010_c: 1.0
})
rxn00173_c.gene_reaction_rule = '272563.120.peg.124'
iCdG698.add_reactions([rxn00173_c])
iCdG698.reactions.rxn00173_c.annotation['sbo'] = 'SBO:0000629'
iCdG698.reactions.rxn00173_c.annotation['seed.reaction'] = 'rxn00173'
iCdG698.reactions.rxn00173_c.annotation['kegg.reaction'] = 'R00230'
iCdG698.reactions.rxn00173_c.annotation['bigg.reaction'] = ['PTA','PTAr']

# Acetylphosphate --> Acetate (already in place: rxn00225_c)

#------------------------------------------------------------------------------#

# Ornithine conversion to alanine

cpd02448_c = cobra.Metabolite(
    'cpd02448_c',
    formula='C5H13N2O2',
    name='2,4-diaminopentanoate',
    compartment='cytosol')
cpd02448_c.annotation['seed.compound'] = 'cpd02448'
cpd02448_c.annotation['kegg.compound'] = 'C03943'

gene = cobra.Gene('272563.120.peg.471')

# D-ornithine <--> 2,4-diaminopentanoate
rxn01780_c = cobra.Reaction('rxn01780_c')
rxn01780_c.name = 'Ornithine aminomutase'
rxn01780_c.lower_bound = -1000.
rxn01780_c.upper_bound = 1000.
rxn01780_c.add_metabolites({
    cpd00064_c: -1.0,
    cpd02448_c: 1.0
})
rxn01780_c.gene_reaction_rule = '272563.120.peg.471'
iCdG698.add_reactions([rxn01780_c])
iCdG698.reactions.rxn01780_c.annotation['kegg.reaction'] = 'R02461'
iCdG698.reactions.rxn01780_c.annotation['sbo'] = 'SBO:0000629'

iCdG698.genes.get_by_id('272563.120.peg.471').name = 'D-Ornithine 4,5-aminomutase E subunit (EC 5.4.3.5) | oraE'
iCdG698.genes.get_by_id('272563.120.peg.471').annotation['sbo'] = 'SBO:0000243'
iCdG698.genes.get_by_id('272563.120.peg.471').annotation['patric'] = 'fig|272563.120.peg.471'
iCdG698.genes.get_by_id('272563.120.peg.471').annotation['refseq'] = 'AJP10141.1'
iCdG698.genes.get_by_id('272563.120.peg.471').annotation['refseq.locustag'] = 'CDIF630_00574'
iCdG698.genes.get_by_id('272563.120.peg.471').annotation['ccds'] = 'ATGTTAAAGGAAAATAAAAAATTGGATATAGACTATATATTAAAAGATTTGGATAAATATAAACCAAAACGAAGAGGTTGGGTTTGGAGAGAACACTTGGAAAATCTTGAAATGGGACCTTTTAAATATAAAGACTGTACAAAACCTTTAAAGAAGAGTGTAGGGCTACCATCATCTAAGTATTTCAATAATATAGACCCTCAACCAAGTCCAGTTATTACTACTGAAATAGCTTCTGGTAGATTTGAGGATGATATAAGAAGAATGAGAATGGCAGCTCATCATGGAGCTGACCATCTAATGGTTATCAGAACTGCTGGACAATCACATTTTGATGGGCTTATAGAAGGAACTCCACAAGGTATAGGTGGAGTTCCAATAACAAGAAAACAAGTAAGAGCTCAAAGGAAGGCTCTTGATTTTATAGAAGAAGAAGTTGGTAGACCTATTAATTACCATTCATATGTAAGTGGTGTTGCTGGTCCTGAAGTAGCAGTAATGTTTGCTGAGGAAGGAGTCAATGGAGCACATCAAGACCCACAATATAATGTTCTGTATAGGAATATAAACATGGTAAGGTCTTTTGTAGATGCTTGTGAGGCAAAGAAAATAATGGCTTTTGCCAATATAGCTCAAATTGATGGAGCACATAATGCTAATGCTACTGCTAGAGAAGCATGGAAGGTAATGCCTGAACTTATGGTTCAACACGCTTTAAACTCTATATTCTCAGAAAAAATAGGTATAGATAAAAGTAATATATGCTTATCAACAGTTCCACCAACAGCACCTCCAGCACCATGTTTAAGAATAGACTTACCTTACGCTGTTGCACTTAGAGAATTATTTAGTGATTATAAGATGAGAGCTCAAATGAATACTAAATATATGGAATCATCAACTAGAGAAGCAACAGTTACTCATGTATTAAACTTATTAACATCAGTATTAACTAGAGCTGACATACAATCAACTATAACACCTGATGAAGGTAGAAATGTACCTTGGCATATATACAATATAGAAGCATGTGATACAGCAAAGCAAGCTTTAGTTGGTATGGATGGGCTTATGGATATGATAGAACTTAAAGATGTCGGTGAATTAAGAGACAAAGCAAGAGAACTTAAAGAAAGAGCTGTATTATATATGGAAGAAATATCTGAAGTTGGTGGATATTTTGAATCTGTGGAACAAGGATTTTTTGTTGATTCTGGAAATTATCCAGAAAGAAATGGTGATGGAATATCTAGAAAAATAAAAGGTGGAGTTGGTGAAGGTACTGTTTATGAAAGAGAAGAAGATTATTTAGCACCTGTAACTGCTCACTTTGGATACAATAATGTAGCTCAATATGATGAAAATGCTATAGATAATCCATCAATGTTAATAGATGGATGTACATTTGAAAATCCAGACAAAATAATATATATTGATGAGCTTGACGATTTTGATAATGTAGAAAATAGACTAAAAGAATCATATGAGTACAGAAATGGAACTAAAATTAAGCCTGAAATGGAATGGTGTGCAGATGGAATTGTAATGATAACAATGATGCTTCCTACTGATAAAAGAACAGCAGAGTTTGCAGCTTTGGAATTTGTCAAAAAAATGAACTTACAAGAAATTGAGGTCATAAGTAGAGAAGTTATGCATGAATCTGAAGGGACTAGAATAGAAGTGAAAGGAAGAGTTCCGTTTGATATAGATTTAAACAATTTAGTAATACCAGAAGAACCTAAGGTACTAACTGATGAAGAAATAAGAGCAGATATAGAAGAAAAACCTATGAAAATAGTTTCAGCTACGGTTGGAGAAGATGAACATTCAGTTGGTTTAAGAGAAATAATAGATATAAAACATGGAGGTATTGAAAAATATGGCATAGAATGCCACTATCTAGGGACTTCTGTACCTGTAGAAAAATTGGTAGATGCAGCGATTGAACTGAATGCAGATGCAATACTTGCATCTACAATAATAAGTCATGATGATATTCACTATAAAAACATGAAAAAATTACACGATTATTGTGTAGAAAAAGGTATAAGAGATAAAGTAATGATTGCGTGTGGAGGAACACAAGTAACACCTGAAATAGCTGTTGAACAAGGCATCGATGCTGGATTTGGAAGAAACAGCAAAGGCATACATGTAGCCACTTTCTTAGTTGAAAAAAGAAGAGAAATGAGTAATAAATAA'

cpd24394_c = cobra.Metabolite(
    'cpd24394_c',
    formula='C5H9NO3',
    name='2-amino-4-ketopentanoate',
    compartment='cytosol')
cpd24394_c.annotation['sbo'] = 'SBO:0000247'
cpd24394_c.annotation['seed.compound'] = 'cpd02448'
cpd24394_c.annotation['metacyc'] = 'CPD-299'

cpd00006_c = iCdG698.metabolites.cpd00006_c # NADP
cpd00005_c = iCdG698.metabolites.cpd00005_c # NADPH

# 2,4-diaminopentanoate <--> 2-amino-4-ketopentanoate
rxn27496_c = cobra.Reaction('rxn27496_c')
rxn27496_c.name = '2,4-diaminopentanoate dehydrogenase'
rxn27496_c.lower_bound = -1000.
rxn27496_c.upper_bound = 1000.
rxn27496_c.add_metabolites({
    cpd00001_c: -1.0,
    cpd02448_c: -1.0,
    cpd00006_c: -1.0,
    cpd00013_c: 1.0,
    cpd00067_c: 1.0,
    cpd24394_c: 1.0,
    cpd00005_c: 1.0
})
rxn27496_c.gene_reaction_rule = '272563.120.peg.467'
iCdG698.add_reactions([rxn27496_c])
iCdG698.reactions.rxn01905_c.gene_reaction_rule = ''
iCdG698.reactions.rxn27496_c.annotation['sbo'] = 'SBO:0000629'
iCdG698.reactions.rxn27496_c.annotation['seed.reaction'] = 'rxn27496'

gene2 = cobra.Gene('272563.120.peg.468')
gene3 = cobra.Gene('645463.3.peg.414')

# 2-amino-4-ketopentanoate <--> alanine
rxn19080_c = cobra.Reaction('rxn19080_c')
rxn19080_c.name = '2-amino-4-oxopentanoate thiolase'
rxn19080_c.lower_bound = -1000.
rxn19080_c.upper_bound = 1000.
rxn19080_c.add_metabolites({
    cpd00010_c: -1.0,
    cpd24394_c: -1.0,
    cpd00022_c: 1.0,
    cpd00035_c: 1.0
})
rxn19080_c.gene_reaction_rule = '272563.120.peg.468 and 272563.120.peg.469'
iCdG698.add_reactions([rxn19080_c])
iCdG698.reactions.rxn19080_c.annotation['sbo'] = 'SBO:0000629'
iCdG698.reactions.rxn19080_c.annotation['seed.reaction'] = 'rxn19080'

iCdG698.genes.get_by_id('272563.120.peg.468').name = '2-amino-4-ketopentanoate thiolase alpha subunit [EC:2.3.1.263] | ortA'
iCdG698.genes.get_by_id('272563.120.peg.468').annotation['sbo'] = 'SBO:0000243'
iCdG698.genes.get_by_id('272563.120.peg.468').annotation['patric'] = 'fig|272563.120.peg.468'
iCdG698.genes.get_by_id('272563.120.peg.468').annotation['kegg'] = 'K21399'
iCdG698.genes.get_by_id('272563.120.peg.468').annotation['refseq'] = 'AJP10139.1'
iCdG698.genes.get_by_id('272563.120.peg.468').annotation['refseq.locustag'] = 'CDIF630_00572'
iCdG698.genes.get_by_id('272563.120.peg.468').annotation['ccds'] = 'ATGGATGCTAAAATTAATGATTGGGTTATTATACACAATATAGTATTGACTCCAGAAGAAAGAGCTCCACAAGTTCCAGAGGATACTAAGAAAGTATCCTTAGAAATGTGGGTTAAAGGTTTTATACAAAGTGATGCTTCTATAGGCGACTTAGTAGAAGTTAAAACAATTACAGGTAGACTTGTAAAAGGTAATTTACTTAAAGTAAATCCATATTATACTCATGATTATGGTAAATGTATACCAGAGTTACTCCAAATAGGAATCCAAGCTAAAGAAATATTATTTGGAGGTGTATATAATGAATAA'

iCdG698.genes.get_by_id('272563.120.peg.469').name = '2-amino-4-ketopentanoate thiolase beta subunit [EC:2.3.1.263] | ortA'
iCdG698.genes.get_by_id('272563.120.peg.469').annotation['sbo'] = 'SBO:0000243'
iCdG698.genes.get_by_id('272563.120.peg.469').annotation['patric'] = 'fig|272563.120.peg.469'
iCdG698.genes.get_by_id('272563.120.peg.469').annotation['kegg'] = 'K21400'
iCdG698.genes.get_by_id('272563.120.peg.469').annotation['refseq'] = 'AJP10139.1'
iCdG698.genes.get_by_id('272563.120.peg.469').annotation['refseq.locustag'] = 'CDIF630_00572'
iCdG698.genes.get_by_id('272563.120.peg.469').annotation['ccds'] = 'ATGAATAATTCAACATCAATAAAAGATATGAGTTATCAAGCTGTTATGGGAAGAAATAACGAAATAATGAAAAATGCAATCGGATTGGATTATTCTTCTTTTGAACAAGAGGGAATAGGGTTTGATTATGAGAAAATGATGAGTGAAACAGGATATACATTACAAGATATTGAAGCAATTCAATCACAGTATGCTGTTGGAAATACTCCTCTAATAGAGCTTAAAAATTTGACTAAATTAGCTAGAAAATGTGCTAAAGAAGGTAAAGGAGCTAAAATATTTGTAAAAGATGAAGCAATGAATGCATCTGGTAGTTTTAAAGCAAGAAGAGCAGCTACTGCAGTATATCATGCCAAACAAATGGGATATAAAGGTGTAATAGCAGCTACCAGTGGTAACTATGGTGCAGCTGTAGCATCACAAGCGGCAATGCAGGGATTAAAATGTATAATAGTTCAAGAATGTTATGATTCTAATGGTGTAGGTCAACCTGAAATTATAGAAAAAGCACGAAAATGTGAGGCATATGGAGCAGAAGTTGTTCAGTTAACTGTTGGACCAGAGTTATTTTATACGTTCTTAGTTTTATTAGAAGAAACAGGCTATTTTAACGCATCATTATATTCACCATTTGGAATTGCAGGTGTAGAAACTTTAGGTTATGAGTTAGCTATCCAATTTAGAGAAAAGTATAAAAAAGACCCTGATATAGTTGTTTGTACAAATGCAGGTGGTGGTAATCTTACTGGAACTGCAAGAGGATTGATAAAAGCTGGTTCCATTAATACTAAAGTAGTAGGAGCAAGTGTAGATTTAAAAGGATTACACATGGCTAGTGATAATCAATTTAATAAAAAATCATTTACAACAGGACATACAGGTTTTGGGATACCATACTGTACATGGCCTGATAGGTCAGATGTACCTAGGTCTGCAGCTAGACCACTTAGATATATGGATAGATATGTATTAGTAAAACAAGGAGAAGTTTTTTATACAACTGAATTATTGGCACAGCTAGAAGGTATAGAAAGAGGTCCAGCAGGAAATACATCTTTAGCAGCAGCATTCTCTTTAGCTCAAGAGTTAGATGAGGATAAAGTTATAGTGGTACAAGAAACAGAATATACAGGCGCAGGTAAACATATATGTCCACAGCTTACTTTTGCAAGAGAAAATGGAATAGATATAAAATTTGGAAATCCAAGAGAAGAAATTGCAGGAGTAAACTTAATATTACCAGAAAGACCAGAATTATTGAAATGTGTTGATATTGATATGAATAAAATAAGAAAGTCATTTATAAAAAATTGCATATCTAATAATCACATTGATAATATTGATAAGCTAAGTAATCGAGATATAGAATTTTTAATGGAAGAAGTAAAAAGTTCAAGAGATTTTGTTGTTGATGTATTAAATAATTTATAG'

#------------------------------------------------------------------------------#

# Correct p-Cresol metabolism
iCdG698.reactions.rxn07124_c.remove_from_model(remove_orphans=True)
iCdG698.reactions.ID008_c.remove_from_model(remove_orphans=True)
iCdG698.reactions.EX_cpd03170_e.remove_from_model(remove_orphans=True)
iCdG698.reactions.ID004_c.remove_from_model(remove_orphans=True)

iCdG698.reactions.rxn05061_c.gene_reaction_rule = '272563.120.peg.166 and 272563.120.peg.167 and 272563.120.peg.168'

iCdG698.reactions.ID005_c.id = 'rxn29230_c'
iCdG698.reactions.rxn29230_c.annotation['seed.reaction'] = 'rxn29230'
iCdG698.reactions.ID009_c.id = 'rxn08184_c'
iCdG698.reactions.rxn08184_c.annotation['seed.reaction'] = 'rxn08184'

# Correct phenylpropanoate transport
iCdG698.reactions.ID007_c.remove_from_model(remove_orphans=False)
cpd03343_c = iCdG698.metabolites.cpd03343_c # Phenylpropanoate
cpd03343_e = iCdG698.metabolites.cpd03343_e # Phenylpropanoate

rxn10133_c = cobra.Reaction('rxn10133_c')
rxn10133_c.name = 'Phenylpropanoate hydrogen symport'
rxn10133_c.lower_bound = -1000.
rxn10133_c.upper_bound = 1000.
rxn10133_c.add_metabolites({
    cpd00067_c: -1.0,
    cpd03343_c: -1.0,
    cpd00067_c: 1.0,
    cpd03343_e: 1.0
})
iCdG698.add_reactions([rxn10133_c])
iCdG698.reactions.rxn10133_c.annotation['sbo'] = 'SBO:0000629'
iCdG698.reactions.rxn10133_c.annotation['seed.reaction'] = 'rxn10133'
iCdG698.reactions.rxn10133_c.annotation['bigg.reaction'] = ['PPPNt2','PPPNt2r','PPPNt2rpp']


cobra.io.save_json_model(iCdG698, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG698_v1.1.sbml')
cobra.io.write_sbml_model(iCdG698, '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG698_v1.1.json')


In [6]:
len(iCdR700.genes)

703

In [8]:
# BHI
bhi = ['cpd00001_e', # water
'cpd00035_e', # alanine
'cpd00041_e', # aspartic acid
'cpd00023_e', # glutamic acid
'cpd00119_e', # histidine
'cpd00107_e', # leucine
'cpd00060_e', # methionine
'cpd00161_e', # threonine
'cpd00069_e', # tyrosine
'cpd00084_e', # cysteine
'cpd00033_e', # glycine
'cpd00322_e', # isoleucine
'cpd00066_e', # phenylalanine
'cpd00054_e', # serine
'cpd00065_e', # tryptophan
'cpd00156_e', # valine
'cpd00220_e', # riboflavin
'cpd00644_e', # pantothentate(B5)
'cpd00393_e', # folate(B9)
'cpd00133_e', # niacin(PP)
'cpd00263_e', # Pyridoxine
'cpd00104_e', # Biotin
'cpd00149_e', # Cobalt
'cpd00971_e', # sodium
'cpd00099_e', # Chloride
'cpd00205_e', # potassium
'cpd00009_e', # phosphate
'cpd00063_e', # calcium
'cpd00254_e', # magnesium
'cpd10515_e', # Fe2+
'cpd00030_e', # Mn2+
'cpd00242_e', # Carbonate
'cpd00226_e', # hypoxanthine
'cpd01242_e', # thyminose
'cpd00307_e', # cytosine
'cpd00092_e', # uracil
'cpd00117_e', # D-Alanine
'cpd00067_e', # H+
'cpd00567_e', # D-Proline
'cpd00132_e', # L-Asparagine
'cpd00210_e', # Taurine
'cpd00320_e', # D-Aspartate
'cpd03279_e', # Deoxyinosine
'cpd00246_e', # Inosine
'cpd00311_e', # Guanosine
'cpd00367_e', # Cytidine
'cpd00277_e', # Deoxyguanosine
'cpd00182_e', # Adenosine
'cpd00654_e', # Deoxycytidine
'cpd00412_e', # Deoxyuridine
'cpd00438_e', # Deoxyadenosine
'cpd00274_e', # Citrulline
'cpd00186_e', # D-Glutamate
'cpd00637_e', # D-Methionine
'cpd00105_e', # D-Ribose
'cpd00305_e', # Thiamin
'cpd00309_e', # Xanthine
'cpd00098_e', # Choline
'cpd00207_e'] # Guanine

bhi_exch = set(['EX_' + x for x in bhi])
for rxn in iCdR700.exchanges:
    rxn.lower_bound = 0.
    if rxn.id in bhi_exch:
        rxn.lower_bound = -1000.
        
iCdR700.reactions.EX_cpd00053_e.lower_bound = -1000. # L-Glutamine
iCdR700.reactions.EX_cpd00039_e.lower_bound = -1000. # Lysine
iCdR700.reactions.EX_cpd00079_e.lower_bound = -1000. # D-glucose-6-phosphate
iCdR700.reactions.EX_cpd00064_e.lower_bound = -1000. # Ornithine
iCdR700.reactions.EX_cpd00051_e.lower_bound = -1000. # D-arginine

base_essential_bhi = find_essential_genes(iCdR700)
base_essential_bhi = set([x.id for x in base_essential_bhi])

In [9]:
# Solo
iCdR700_solo = riptide.contextualize(model=iCdR700, transcriptome=solo, fraction=0.75)

# Coculture
iCdR700_coculture = riptide.contextualize(model=iCdR700, transcriptome=coculture, fraction=0.75)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 306 from 1317 (76.77% change)
Metabolites pruned to 306 from 1245 (75.42% change)
Flux through the objective DECREASED to ~95.0 from ~124.92 (23.95% change)
Context-specific metabolism correlates with transcriptome (r=0.21, p<0.001 *)

RIPTiDe completed in 17 seconds


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 309 from 1317 (76.54% change)
Metabolites pruned to 309 from 1245 (75.18% change)
Flux through the objective DECREASED to ~97.92 from ~124.92 (21.61% change)
Context-specific metabolism correlates with transcriptome (r=0.128, p=0.024 *)

RIPTiDe completed in 19 seconds



In [26]:
riptide.save_output(riptide_obj=iCdR700_solo, path='/home/mjenior/Desktop/active_projects/Zackular_enterococcus/solo_bhi_new')
riptide.save_output(riptide_obj=iCdR700_coculture, path='/home/mjenior/Desktop/active_projects/Zackular_enterococcus/coculture_bhi_new')

In [10]:
iCdR700_solo.model

Name,iCdR700_riptide
Memory address,0x07fcd4ca7d290
Number of metabolites,306
Number of reactions,306
Number of groups,0
Objective expression,0.0 + 1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [11]:
len(iCdR700_solo.model.genes)

220

In [12]:
iCdR700_coculture.model

Name,iCdR700_riptide
Memory address,0x07fcd31b8abd0
Number of metabolites,309
Number of reactions,309
Number of groups,0
Objective expression,0.0 + 1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular, periplasm"


In [13]:
len(iCdR700_coculture.model.genes)

218

In [37]:
from random import sample
import scipy

doubling = round((1.0 / iCdR700_solo.model.slim_optimize()) * 3200.0, 2)
print('Solo doubling time: ' + str(doubling) + ' minutes')

doubling = round((1.0 / iCdR700_coculture.model.slim_optimize()) * 3200.0, 2)
print('Coculture doubling time: ' + str(doubling) + ' minutes')

pvals = []
maxRange = len(list(iCdR700_solo.flux_samples['biomass'])) * 10
for x in range(1, maxRange):
        solo_sample = sample(list(iCdR700_solo.flux_samples['biomass']), 10)
        coculture_sample = sample(list(iCdR700_coculture.flux_samples['biomass']), 10)
        pvals.append(scipy.stats.wilcoxon(x=solo_sample, y=coculture_sample)[1])
print('p-value', round(numpy.median(pvals),3))

Solo doubling time: 33.68 minutes
Coculture doubling time: 32.68 minutes
p-value 0.285


In [38]:
# Distinct reactions - BHI
rxns1 = set([x.id for x in iCdR700_solo.model.reactions])
rxns2 = set([x.id for x in iCdR700_coculture.model.reactions])

solo_only_rxns = rxns1.difference(rxns2)
print('Solo reactions:', len(solo_only_rxns))
for x in solo_only_rxns: print(x, iCdR700.reactions.get_by_id(x).name)

coculture_only_rxns = rxns2.difference(rxns1)
print('\nCoculture-only reactions:', len(coculture_only_rxns))
for x in coculture_only_rxns: print(x, iCdR700.reactions.get_by_id(x).name)   

Solo reactions: 15
EX_cpd00023_e L-Glutamate exchange
rxn05204_c Guanosine hydrogen symport
rxn01286_c D-Glyceraldehyde:NAD+ oxidoreductase
rxn05298_c Na+/glutamate symport
rxn08647_c ATP:(R)-glycerate 2-phosphotransferase
EX_cpd00013_e Ammonia exchange
rxn00459_c 2-phospho-D-glycerate hydro-lyase (phosphoenolpyruvate-forming)
rxn05234_c 2'-Deoxyguanosine 5'-triphosphate:oxydized-thioredoxin 2'-oxidoreductase
rxn02380_c beta-D-Glucose 6-phosphate ketol-isomerase
EX_cpd00001_e H2O exchange
rxn08686_c H2O transport via diffusion
rxn05467_c CO2 transport via diffusion
rxn05466_c Ammonia diffusion
EX_cpd00011_e CO2 exchange
EX_cpd00311_e Guanosine exchange

Coculture-only reactions: 18
EX_cpd00186_e D-Glutamate exchange
EX_cpd00067_e H+ exchange
rxn00504_c (S)-1-pyrroline-5-carboxylate:NADP+ oxidoreductase
EX_cpd00277_e Deoxyguanosine exchange
rxn01649_c Xanthosine:orthophosphate ribosyltransferase
rxn05203_c Guanine symport
rxn00931_c L-Proline:NADP+ 5-oxidoreductase
rxn01297_c hypoxanthi

In [39]:
# Shared exchanges
rxns1 = set([x.id for x in iCdR700_solo.model.exchanges])
rxns2 = set([x.id for x in iCdR700_coculture.model.exchanges])
shared_exch = rxns1.intersection(rxns2)

print('Shared exchange reactions:', len(shared_exch))
for x in shared_exch: print(x, iCdR700.reactions.get_by_id(x).name)


Shared exchange reactions: 46
EX_cpd19585_e 2-Methylbutyrate exchange
EX_cpd00063_e Ca2+ exchange
EX_cpd00039_e L-Lysine exchange
EX_cpd00107_e L-Leucine exchange
EX_cpd00133_e Nicotinamide exchange
EX_cpd00149_e Co2+ exchange
EX_cpd00035_e L-Alanine exchange
EX_cpd00069_e L-Tyrosine exchange
EX_cpd00412_e Deoxyuridine exchange
EX_cpd00971_e Na+ exchange
EX_cpd00030_e Mn2+ exchange
EX_cpd00104_e Biotin exchange
EX_cpd00254_e Mg exchange
EX_cpd00117_e D-Alanine exchange
EX_cpd00065_e L-Tryptophan exchange
EX_cpd00644_e Pantothenate exchange
EX_cpd00066_e L-Phenylalanine exchange
EX_cpd00161_e L-Threonine exchange
EX_cpd00099_e Cl- exchange
EX_cpd00054_e L-Serine exchange
EX_cpd00654_e Deoxycytidine exchange
EX_cpd10515_e Fe2+ exchange
EX_cpd00246_e Inosine exchange
EX_cpd00263_e Pyridoxol exchange
EX_cpd00041_e L-Aspartate exchange
EX_cpd00060_e L-Methionine exchange
EX_cpd00393_e Folate exchange
EX_cpd00009_e Phosphate exchange
EX_cpd00092_e Uracil exchange
EX_cpd00220_e Riboflavin exc

In [40]:
# Distinct metabolites - BHI
cpds1 = set([x.id for x in iCdR700_solo.model.metabolites])
cpds2 = set([x.id for x in iCdR700_coculture.model.metabolites])

solo_only_cpds = cpds1.difference(cpds2)
print('Solo culture-only metabolites:', len(solo_only_cpds))
for x in solo_only_cpds: print(x, iCdR700.metabolites.get_by_id(x).name)

coculture_only_cpds = cpds2.difference(cpds1)
print('\nCoculture-only metabolites:', len(coculture_only_cpds))
for x in coculture_only_cpds: print(x, iCdR700.metabolites.get_by_id(x).name) 

Solo culture-only metabolites: 7
cpd00011_e CO2
cpd00311_e Guanosine
cpd00013_e Ammonia
cpd00001_e H2O
cpd00482_c 2-Phospho-D-glycerate
cpd00223_c Glycerate
cpd00023_e L-Glutamate

Coculture-only metabolites: 10
cpd00277_c Deoxyguanosine
cpd00277_e Deoxyguanosine
cpd00165_c Hydroxylamine
cpd00207_c Guanine
cpd00165_e Hydroxylamine
cpd00309_c Xanthine
cpd00207_e Guanine
cpd02431_c 1-Pyrroline-5-carboxylate
cpd00186_e D-Glutamate
cpd01217_c Xanthosine


In [71]:
# Distinct genes - BHI
genes1 = set([x.id for x in iCdR700_solo.model.genes])
genes2 = set([x.id for x in iCdR700_coculture.model.genes])

solo_only_genes = genes1.difference(genes2)
print('Solo-only genes:', len(solo_only_genes))
for x in solo_only_genes: print(x, iCdR700.genes.get_by_id(x).name)

coculture_only_genes = genes2.difference(genes1)
print('\nCoculture-only genes:', len(coculture_only_genes))
for x in coculture_only_genes: print(x, iCdR700.genes.get_by_id(x).name) 

Solo-only genes: 6
645463.3.peg.2226 Proton/sodium-glutamate symport protein
645463.3.peg.1749 Sodium/glutamate symporter
645463.3.peg.3215 Enolase (EC 4.2.1.11)
645463.3.peg.2261 Aldehyde dehydrogenase (EC 1.2.1.3)
645463.3.peg.1272 pyridoxal kinase (EC 2.7.1.35)
645463.3.peg.2741 Sodium/glutamate symporter

Coculture-only genes: 4
645463.3.peg.1483 phosphoribosyl-AMP cyclohydrolase (EC 3.5.4.19)
645463.3.peg.1423 Pyrroline-5-carboxylate reductase (EC 1.5.1.2)
645463.3.peg.3335 Pyrroline-5-carboxylate reductase (EC 1.5.1.2)
645463.3.peg.2222 hydroxylamine reductase (EC 1.7.99.1)


In [72]:
# Differential gene essentiality
solo_threshold = iCdR700_solo.model.slim_optimize() * 0.05
solo_essential_bhi = find_essential_genes(iCdR700_solo.model, threshold=solo_threshold)
solo_essential_bhi = set([x.id for x in solo_essential_bhi]).difference(base_essential_bhi)

coculture_threshold = iCdR700_coculture.model.slim_optimize() * 0.05
coculture_essential_bhi = find_essential_genes(iCdR700_coculture.model, threshold=coculture_threshold)
coculture_essential_bhi = set([x.id for x in coculture_essential_bhi]).difference(base_essential_bhi)

solo_essential_bhi_only = solo_essential_bhi.difference(coculture_essential_bhi)
coculture_essential_bhi_only = coculture_essential_bhi.difference(solo_essential_bhi)
essential_bhi = solo_essential_bhi.intersection(coculture_essential_bhi)

both_contexts = solo_essential_bhi.intersection(coculture_essential_bhi)

print('Solo-only essential genes:', len(solo_essential_bhi_only))
for x in solo_essential_bhi_only: print(x, iCdR700.genes.get_by_id(x).name)
print('\nCoculture-only essential genes:', len(coculture_essential_bhi_only))
for x in coculture_essential_bhi_only: print(x, iCdR700.genes.get_by_id(x).name)

Solo-only essential genes: 2
645463.3.peg.1079 Acetate kinase (EC 2.7.2.1)
645463.3.peg.2376 4-hydroxybutyrate:acetyl-CoA CoA transferase (EC 2.3.1.-)

Coculture-only essential genes: 2
645463.3.peg.3217 Triosephosphate isomerase (EC 5.3.1.1)
645463.3.peg.194 NAD-specific glutamate dehydrogenase (EC 1.4.1.2)


In [42]:
print('\nCore essential:', len(base_essential_bhi))
for x in base_essential_bhi: print(x, iCdR700.genes.get_by_id(x).name)


Core essential: 72
645463.3.peg.48 2-C-methyl-D-erythritol 4-phosphate cytidylyltransferase (EC 2.7.7.60)
645463.3.peg.2106 Glutaminyl-tRNA synthetase (EC 6.1.1.18)
645463.3.peg.1828 4-hydroxy-3-methylbut-2-enyl diphosphate reductase (EC 1.17.1.2)
645463.3.peg.57 thymidylate synthase (EC 2.1.1.45)
645463.3.peg.682 Phenylalanyl-tRNA synthetase beta chain (EC 6.1.1.20)
645463.3.peg.2802 Phosphoenolpyruvate-protein phosphotransferase of PTS system (EC 2.7.3.9) | ptsI
645463.3.peg.3143 PTS system, trehalose-specific IIB component (EC 2.7.1.69)
645463.3.peg.2479 Glycyl-tRNA synthetase beta chain (EC 6.1.1.14)
645463.3.peg.1842 3-dehydroquinate synthase (EC 4.2.3.4)
645463.3.peg.2042 Substrate-specific component TrpP of tryptophan ECF transporter
645463.3.peg.2701 UDP-N-acetylmuramoylalanine--D-glutamate ligase (EC 6.3.2.9)
645463.3.peg.2703 UDP-N-acetylmuramoyl-tripeptide--D-alanyl-D-alanine ligase
645463.3.peg.3609 4-diphosphocytidyl-2-C-methyl-D-erythritol kinase (EC 2.7.1.148)
645463.3.

In [43]:
# Differential reaction essentiality
solo_essential_bhi = find_essential_reactions(iCdR700_solo.model)
solo_essential_bhi = set([x.id for x in solo_essential_bhi]).difference(base_essential_bhi)
coculture_essential_bhi = find_essential_reactions(iCdR700_coculture.model)
coculture_essential_bhi = set([x.id for x in coculture_essential_bhi]).difference(base_essential_bhi)
solo_essential_bhi_only = solo_essential_bhi.difference(coculture_essential_bhi)
coculture_essential_bhi_only = coculture_essential_bhi.difference(solo_essential_bhi)
essential_bhi = solo_essential_bhi.intersection(coculture_essential_bhi)

print('Solo only', len(solo_essential_bhi_only))
for x in solo_essential_bhi_only: print(x, iCdR700.reactions.get_by_id(x).name)   
print('\nCoculture only', len(coculture_essential_bhi_only))
for x in coculture_essential_bhi_only: print(x, iCdR700.reactions.get_by_id(x).name)   


Solo only 15
EX_cpd00023_e L-Glutamate exchange
rxn13713_c 4-hydroxybutyrate CoA-transferase
rxn05204_c Guanosine hydrogen symport
rxn01236_c ATP:butanoate 1-phosphotransferase
rxn05298_c Na+/glutamate symport
rxn00225_c ATP:acetate phosphotransferase
rxn03641_c acetyl-CoA: 4-hydroxybutanoate CoA transferase
EX_cpd00029_e Acetate exchange
rxn01345_c Sedoheptulose 1,7-bisphosphate 1-phosphohydrolase
rxn05234_c 2'-Deoxyguanosine 5'-triphosphate:oxydized-thioredoxin 2'-oxidoreductase
rxn01334_c Sedoheptulose 1,7-bisphosphate D-glyceraldehyde-3-phosphate-lyase
rxn08061_c Acetate transport via proton symport
rxn00193_c glutamate racemase
EX_cpd00311_e Guanosine exchange
rxn00871_c Phosphate butyrltransferase

Coculture only 6
rxn00747_c D-glyceraldehyde-3-phosphate aldose-ketose-isomerase
EX_cpd00277_e Deoxyguanosine exchange
rxn09688_c deoxyguanosine transport in via proton symport (periplasm)
rxn13140_c Nucleoside triphosphatase
rxn00182_c L-glutamate:NAD+ oxidoreductase (deaminating)
rxn

In [44]:
# Exchanges
solo_exch = set([x.id for x in iCdR700_solo.model.exchanges])
coculture_exch = set([x.id for x in iCdR700_coculture.model.exchanges])

solo_exch_only = solo_exch.difference(coculture_exch)
coculture_exch_only = coculture_exch.difference(solo_exch)

print('Solo in BHI only', len(solo_exch_only))
for x in solo_exch_only: print(x, iCdR700.reactions.get_by_id(x).name)   
print('\nCoculture in BHI only', len(coculture_exch_only))
for x in coculture_exch_only: print(x, iCdR700.reactions.get_by_id(x).name)  

Solo in BHI only 5
EX_cpd00023_e L-Glutamate exchange
EX_cpd00013_e Ammonia exchange
EX_cpd00001_e H2O exchange
EX_cpd00011_e CO2 exchange
EX_cpd00311_e Guanosine exchange

Coculture in BHI only 5
EX_cpd00186_e D-Glutamate exchange
EX_cpd00067_e H+ exchange
EX_cpd00277_e Deoxyguanosine exchange
EX_cpd00165_e Hydroxylamine exchange
EX_cpd00207_e Guanine exchange


In [45]:
# Input
for (columnName, columnData) in iCdR700_solo.flux_samples.iteritems():
    if 'EX_' in columnName:
        colMedian = round(numpy.median(columnData.values),3)
        if colMedian < -1e-6:
            substrate = list(iCdR700.reactions.get_by_id(columnName).reactants)[0].name
            print(substrate, columnName, colMedian)

H2O EX_cpd00001_e -284.401
L-Glutamate EX_cpd00023_e -56.63
Na+ EX_cpd00971_e -0.071
L-Arginine EX_cpd00051_e -4.56
Ornithine EX_cpd00064_e -3.905
Fe2+ EX_cpd10515_e -0.071
L-Methionine EX_cpd00060_e -3.848
L-Alanine EX_cpd00035_e -853.638
Riboflavin EX_cpd00220_e -0.078
D-Alanine EX_cpd00117_e -74.813
Mg EX_cpd00254_e -0.071
K+ EX_cpd00205_e -0.071
Thiamin EX_cpd00305_e -0.071
L-Tryptophan EX_cpd00065_e -0.912
Co2+ EX_cpd00149_e -0.071
Cl- EX_cpd00099_e -0.071
Biotin EX_cpd00104_e -0.071
Ca2+ EX_cpd00063_e -0.071
L-Valine EX_cpd00156_e -9.348
L-Lysine EX_cpd00039_e -34.4
Folate EX_cpd00393_e -0.071
L-Isoleucine EX_cpd00322_e -165.436
Mn2+ EX_cpd00030_e -0.071
L-Cysteine EX_cpd00084_e -1.845
Pantothenate EX_cpd00644_e -0.15
Pyridoxol EX_cpd00263_e -0.071
L-Leucine EX_cpd00107_e -19.139
L-Tyrosine EX_cpd00069_e -5.928
L-Phenylalanine EX_cpd00066_e -6.242
L-Histidine EX_cpd00119_e -1.938
L-Asparagine EX_cpd00132_e -9.177
L-Serine EX_cpd00054_e -79.609
L-Threonine EX_cpd00161_e -7.068
L-A

In [46]:
# Output
for (columnName, columnData) in iCdR700_solo.flux_samples.iteritems():
    if 'EX_' in columnName:
        colMedian = round(numpy.median(columnData.values),3)
        if colMedian > 1e-6:
            substrate = list(iCdR700.reactions.get_by_id(columnName).reactants)[0].name
            print(substrate, columnName, colMedian)

Phosphate EX_cpd00009_e 58.132
N-Acetyl-D-glucosamine EX_cpd00122_e 618.77
Hypoxanthine EX_cpd00226_e 683.012
Uracil EX_cpd00092_e 915.062
CO2 EX_cpd00011_e 404.937
Isovalerate EX_cpd05178_e 4.897
2-Methylbutyrate EX_cpd19585_e 150.073
Acetate EX_cpd00029_e 972.575
Ammonia EX_cpd00013_e 387.216
Formate EX_cpd00047_e 1000.0


In [47]:
# Input
for (columnName, columnData) in iCdR700_coculture.flux_samples.iteritems():
    if 'EX_' in columnName:
        colMedian = round(numpy.median(columnData.values),3)
        if colMedian < -1e-6:
            substrate = list(iCdR700.reactions.get_by_id(columnName).reactants)[0].name
            print(substrate, columnName, colMedian)

Na+ EX_cpd00971_e -0.073
L-Arginine EX_cpd00051_e -4.7
Ornithine EX_cpd00064_e -103.569
Fe2+ EX_cpd10515_e -0.073
L-Methionine EX_cpd00060_e -3.966
L-Alanine EX_cpd00035_e -746.186
Riboflavin EX_cpd00220_e -0.081
D-Alanine EX_cpd00117_e -77.109
Mg EX_cpd00254_e -0.073
K+ EX_cpd00205_e -0.073
Thiamin EX_cpd00305_e -0.073
L-Tryptophan EX_cpd00065_e -0.94
Co2+ EX_cpd00149_e -0.073
Cl- EX_cpd00099_e -0.073
Biotin EX_cpd00104_e -0.073
Ca2+ EX_cpd00063_e -0.073
L-Valine EX_cpd00156_e -9.635
L-Lysine EX_cpd00039_e -35.456
Folate EX_cpd00393_e -0.073
L-Isoleucine EX_cpd00322_e -191.142
Mn2+ EX_cpd00030_e -0.073
L-Cysteine EX_cpd00084_e -1.902
Pantothenate EX_cpd00644_e -0.154
Pyridoxol EX_cpd00263_e -0.073
L-Leucine EX_cpd00107_e -223.464
L-Tyrosine EX_cpd00069_e -6.11
L-Phenylalanine EX_cpd00066_e -6.433
L-Histidine EX_cpd00119_e -1.998
L-Asparagine EX_cpd00132_e -9.459
L-Serine EX_cpd00054_e -382.682
L-Threonine EX_cpd00161_e -7.285
L-Aspartate EX_cpd00041_e -76.101
Nicotinamide EX_cpd00133_

In [25]:
# Output
for (columnName, columnData) in iCdR700_coculture.flux_samples.iteritems():
    if 'EX_' in columnName:
        colMedian = round(numpy.median(columnData.values),3)
        if colMedian > 1e-6:
            substrate = list(iCdR700.reactions.get_by_id(columnName).reactants)[0].name
            print(substrate, columnName, colMedian)

Phosphate EX_cpd00009_e 165.666
H+ EX_cpd00067_e 85.343
N-Acetyl-D-glucosamine EX_cpd00122_e 812.11
Hypoxanthine EX_cpd00226_e 432.457
Uracil EX_cpd00092_e 595.37
Isovalerate EX_cpd05178_e 210.099
2-Methylbutyrate EX_cpd19585_e 176.198
Acetate EX_cpd00029_e 862.527
Formate EX_cpd00047_e 1000.0
Hydroxylamine EX_cpd00165_e 490.404
D-Glutamate EX_cpd00186_e 36.994
Guanine EX_cpd00207_e 336.706


In [77]:
# Define minimal medium for each model

# constrained to media condition
minmedia_solo = minimal_medium(iCdR700_solo.model, minimize_components=True, min_objective_value=0.8)
minmedia_coculture = minimal_medium(iCdR700_coculture.model, minimize_components=True, min_objective_value=0.8)

coculture_min_only = set(list(minmedia_coculture.index)).difference(set(list(minmedia_solo.index)))
solo_min_only = set(list(minmedia_solo.index)).difference(set(list(minmedia_coculture.index)))

print('Solo in BHI only', len(solo_min_only))
for x in solo_min_only: print(x, iCdR700.reactions.get_by_id(x).name)   
print('\nCoculture in BHI only', len(coculture_min_only))
for x in coculture_min_only: print(x, iCdR700.reactions.get_by_id(x).name)  

Solo in BHI only 5
EX_cpd00311_e Guanosine exchange
EX_cpd00035_e L-Alanine exchange
EX_cpd00412_e Deoxyuridine exchange
EX_cpd00079_e D-glucose-6-phosphate exchange
EX_cpd00277_e Deoxyguanosine exchange

Coculture in BHI only 4
EX_cpd00064_e Ornithine exchange
EX_cpd00246_e Inosine exchange
EX_cpd00186_e D-Glutamate exchange
EX_cpd00009_e Phosphate exchange
